In [1]:
import requests
import json
import numpy as np
import pandas as pd
from pandas import json_normalize # easy JSON -> pd.DataFrame

In [2]:
auth_endpoint = "https://auth.emsicloud.com/connect/token" # auth endpoint

client_id = "n7gpmbt9bmj5f4mg" # replace 'your_client_id' with your client id from your api invite email
client_secret = "tYx0dFGv" # replace 'your_client_secret' with your client secret from your api invite email
scope = "emsi_open" # ok to leave as is, this is the scope we will used

payload = "client_id=" + client_id + "&client_secret=" + client_secret + "&grant_type=client_credentials&scope=" + scope # set credentials and scope
headers = {'content-type': 'application/x-www-form-urlencoded'} # headers for the response
access_token = json.loads((requests.request("POST", auth_endpoint, data=payload, headers=headers)).text)['access_token'] # grabs request's text and loads as JSON, then pulls the access token from that


In [3]:
def extract_skills_list():
  all_skills_endpoint = "https://emsiservices.com/skills/versions/latest/skills" # List of all skills endpoint
  auth = "Authorization: Bearer " + access_token # Auth string including access token from above
  headers = {'authorization': auth} # headers
  response = requests.request("GET", all_skills_endpoint, headers=headers) # response
  response = response.json()['data'] # the data

  all_skills_df = pd.DataFrame(json_normalize(response)); # Where response is a JSON object drilled down to the level of 'data' key
  return all_skills_df

df = extract_skills_list()

In [4]:
df = df[['name','type.name']]
df = df.rename(columns={'type.name': 'type'})

In [5]:
df = df[df.type!='Certification']

In [6]:
df.to_csv("/Users/ivowings/Sync/Thesis/Datasources/emsi_skills.csv",ind)